In [6]:
''' Libraries ''' 
import requests
import os
import pandas as pd
from fuzzywuzzy import process, fuzz
from urllib.request import urlopen

In [3]:
def tag(nombre, api):
    name = nombre + '_'+api
    return name
tabla = pd.read_excel(r'../Merge.xlsx')
#tabla.drop(columns='Unnamed: 0', inplace=True)
# ALPHA Vantage
columnas_Alpha  =['fiscalDateEnding', 'reportedCurrency']+ tabla.tag_alpha.tolist()
tabla.head()

,tag_tiingo,desc_tiingo,tag_alpha,desc_alpha,tag_finmodprep,tag_finn,tag_eod,tag_sharadar
0,totalAssets,Total Assets,totalAssets,"Assets, Total",totalAssets,totalAssets,totalAssets,ASSETS
1,totalLiabilities,Total Liabilities,totalLiabilities,"Liabilities, Total",totalLiabilities,totalLiabilities,totalLiab,LIABILITIES
2,sga,"Selling, General & Administrative",sellingGeneralAndAdministrative,"Selling, General and Administrative Expense, T...",sellingGeneralAndAdministrativeExpenses,sgaExpense,sellingGeneralAdministrative,SGNA
3,rnd,Research & Development,researchAndDevelopment,"Research and Development Expense, Total",researchAndDevelopmentExpenses,researchDevelopment,researchDevelopment,RND
4,ebitda,EBITDA,ebitda,"Earnings Before Interest, Taxes, Depreciation,...",ebitda,ebit,ebitda,EBITDA


In [9]:
def alpha_vantage(ticker, period):
    def tag(nombre, api):
        name = nombre + '_'+api
        return name
    tabla = pd.read_excel(r'../Merge.xlsx')
    columnas_Alpha = ['fiscalDateEnding', 'reportedCurrency'] + tabla.tag_alpha.tolist()

    url = f'https://www.alphavantage.co/query?function=Cash_flow&symbol={ticker}&apikey=6DWJBYP02X4S13GL'
    r = requests.get(url)
    data = r.json()
    df_cash = pd.DataFrame(data[period])
    

    url = f'https://www.alphavantage.co/query?function=Balance_Sheet&symbol={ticker}&apikey=6DWJBYP02X4S13GL'
    r = requests.get(url)
    data = r.json()
    df_balance = pd.DataFrame(data[period])

    url = f'https://www.alphavantage.co/query?function=Income_Statement&symbol={ticker}&apikey=6DWJBYP02X4S13GL'
    r = requests.get(url)
    data = r.json()
    df_income = pd.DataFrame(data[period])
    
    df_alpha = pd.concat([ df_cash, df_balance , df_income], axis=1)
    df_alpha = df_alpha[columnas_Alpha]
    df_alpha = df_alpha.loc[:, ~df_alpha.columns.duplicated()]
    df_alpha.fiscalDateEnding = pd.to_datetime(df_alpha.fiscalDateEnding)
    df_alpha['fiscalDateEnding'] = df_alpha.fiscalDateEnding.dt.strftime('%Y-%m')
    dict_alpha = {}
    for i in range(len(df_alpha.columns)):
        dict_alpha[df_alpha.columns[i]] = tag(df_alpha.columns[i],'alpha' )
    df_alpha.rename(columns=dict_alpha, inplace=True)
    df_alpha.insert(0,'ticker_alpha' , f'{ticker}')
    '''
    ''' 
    
    return df_alpha


In [10]:
df_alpha = alpha_vantage('AAPL','annualReports')
df_alpha

,ticker_alpha,fiscalDateEnding_alpha,reportedCurrency_alpha,totalAssets_alpha,totalLiabilities_alpha,sellingGeneralAndAdministrative_alpha,researchAndDevelopment_alpha,ebitda_alpha,deferredRevenue_alpha,costOfRevenue_alpha,...,inventory_alpha,operatingIncome_alpha,comprehensiveIncomeNetOfTax_alpha,currentAccountsPayable_alpha,cashAndShortTermInvestments_alpha,commonStock_alpha,currentNetReceivables_alpha,capitalExpenditures_alpha,totalShareholderEquity_alpha,changeInOperatingAssets_alpha
0,AAPL,2022-09,USD,352755000000,302083000000,25094000000,26251000000,130207000000,20312000000,248640000000,...,4946000000,119437000000,88531000000,64115000000,48304000000,64849000000,60932000000,10708000000,50672000000,14358000000
1,AAPL,2021-09,USD,351002000000,287912000000,21973000000,21914000000,123136000000,19512000000,234954000000,...,6580000000,108949000000,95249000000,54763000000,62639000000,57365000000,51506000000,11085000000,63090000000,24712000000
2,AAPL,2020-09,USD,323888000000,258549000000,19916000000,18752000000,81020000000,16843000000,189475000000,...,4061000000,66288000000,57453000000,42296000000,90943000000,50779000000,37445000000,7309000000,65339000000,1245000000
3,AAPL,2019-09,USD,338516000000,248028000000,18245000000,16217000000,81860000000,13622000000,180027000000,...,4106000000,63930000000,58037000000,46236000000,100557000000,45174000000,45804000000,10495000000,90488000000,-3760000000
4,AAPL,2018-09,USD,365725000000,258578000000,16705000000,14236000000,87046000000,14766000000,163756000000,...,3956000000,70898000000,56505000000,55888000000,25913000000,40201000000,48995000000,13313000000,107147000000,12927000000
